## Multilayer Perceptron (MLP)

- Camadas intermediárias
- Arquitetura feedforward
- Aprendizagem por retropropagação (backpropagation)

<img src="images/MLP.png">

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [12]:
# importando dataset
dataset = pd.read_csv("frutas.csv")
dataset.head()

,ph,peso(kg),diametro(cm),fruta
0,2.980056,0.059878,8.303492,0
1,2.619839,0.059534,9.563558,0
2,2.983526,0.040029,8.921555,0
3,2.827375,0.046539,5.919455,0
4,2.979685,0.055912,9.392527,0


In [13]:
# extraindo características (x)
x = np.array(dataset.loc[:, ["ph", "peso(kg)", "diametro(cm)"]])
x

array([[ 2.98005606e+00,  5.98781498e-02,  8.30349159e+00],
       [ 2.61983873e+00,  5.95344254e-02,  9.56355780e+00],
       [ 2.98352600e+00,  4.00289555e-02,  8.92155540e+00],
       [ 2.82737493e+00,  4.65386710e-02,  5.91945531e+00],
       [ 2.97968528e+00,  5.59116887e-02,  9.39252729e+00],
       [ 2.99625546e+00,  5.36535158e-02,  7.63980642e+00],
       [ 3.30110957e+00,  4.72463817e-02,  8.08016791e+00],
       [ 3.42557126e+00,  5.15197758e-02,  6.19331529e+00],
       [ 2.78253640e+00,  3.49039649e-02,  6.81114797e+00],
       [ 3.55385466e+00,  3.74214658e-02,  7.66047212e+00],
       [ 3.52264228e+00,  7.03788412e-02,  8.67622907e+00],
       [ 2.84906888e+00,  6.15340605e-02,  7.99957211e+00],
       [ 2.88285328e+00,  4.35992624e-02,  8.89179211e+00],
       [ 3.28386152e+00,  5.01243049e-02,  9.01991579e+00],
       [ 2.74989284e+00,  6.18838284e-02,  7.91505903e+00],
       [ 3.47640560e+00,  4.55800761e-02,  9.87091082e+00],
       [ 2.31414672e+00,  4.36382278e-02

In [14]:
# extraindo resultado
d = np.array(dataset.loc[:,"fruta"])
d

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

#### Mapeando saídas:

Como nossa rede possui 3 neurônios de saída, vamos mapear qual será a saída esperada de cada neurônio para cada fruta

Fruta | Neurônio 0 | Neurônio 1 | Neurônio 2
---|---|---|---
Maçã  | 1 | 0 | 0
Limão | 0|1|0
Melão|0|0|1

In [15]:
# Mapeando classe para vetor
def vetorizar(d):
    saidas = []
    for fruta in d:
        if fruta == 0:
            saidas.append(np.array([1, 0, 0]))
        elif fruta == 1:
            saidas.append(np.array([0, 1, 0]))
        else:
            saidas.append(np.array([0, 0, 1]))

    return np.array(saidas)

# Mapeando vetor para classe
def classificar(vetor):
    classes = []
    for y in vetor:
        maximo = np.argmax(y)
        classes.append(maximo)
        
    return classes

In [16]:
d = vetorizar(d)

# Embaralhando os dados
shuffle = np.random.permutation(len(x))
x = x[shuffle]
d = d[shuffle]

In [17]:
# Separando dados de treino (80%) e teste (20%)
limite = int(len(x) * 0.8)

x_treino = x[0:limite]
x_teste = x[limite: ]

d_treino = d[0:limite]
d_teste = d[limite: ]

In [ ]:
class MLP():
    # Construtor
    def __init__(self, lr, e, neurons):
        self.lr = lr # Taxa de aprendizado 
        self.e = e # tolerância
        self.neurons = neurons # Quantidade de neurônios por camadas
        
    # Calcula a sigmoid de um valor    
    def sigmoid(self, valor):
        return (1/(1 + math.e ** (-valor)))

    # Calcula a derivada da função sigmoid
    def sigmoid_deriv(self, valor):
        sig = self.sigmoid(valor)
        return sig*(1 - sig)

    # Ativa as saídas do neurônio
    def activate(self, valor):
        return self.sigmoid(valor)
    
    # Calcular a derivada da função de ativação
    def deriv(self, valor):
        return self.sigmoid_deriv(valor)

    # Calcula a diferença entre o valor real e o valor predito
    def evaluate(self, target, predicted):
        return (target - predicted)

    # Calcula a soma ponderada das entradas pelo peso
    def predict(self, input_data, weights):
        return np.dot(input_data, weights).reshape(1, -1)
    
    
    def train(self, x, d):
        # Inicializa aleatoriamente os pesos para cada camada da rede
        # Pesos da camada de entrada para a 1ª camada oculta
        self.w1 = np.random.random((x.shape[1] + 1, self.neurons[0])) 
        
        # Pesos da 1ª para a 2ª camada oculta
        self.w2 = np.random.random((self.neurons[0], self.neurons[1]))

        # Pesos da 2ª camada oculta para a camada de saída
        self.w3 = np.random.random((self.neurons[1], self.neurons[2]))
        
        epoch = 0
        last_mse = np.inf # Inicializa o MSE anterior como infinito
        self.total_mse = [] # Armazena os valores de MSE para cada época
        self.bias = -1
        
        # Enquanto a diferença entre m mse_anterior e o mse_atual for maior que 'e' continua o processo 
        while True:
            mse = 0
            for xi, target in zip(x,d):
                # Adiciona o bias à entrada
                input_value = np.insert(xi, 0, self.bias)

                # Forward pass: Calcula as ativações da rede
                i1 = self.predict(input_value, self.w1)   # Soma ponderada para a 1ª camada
                y1 = self.activate(i1)                    # Ativação da 1ª camada
                i2 = self.predict(y1, self.w2)            # Soma ponderada para a 2ª camada
                y2 = self.activate(i2)                    # Ativação da 2ª camada
                i3 = self.predict(y2, self.w3)            # Soma ponderada para a camada de saída
                y3 = self.activate(i3)                    # Ativação da camada de saída
                
                # Calcula o erro atual
                current_error = self.evaluate(target, y3)
                mse+=(current_error ** 2)    # Soma o erro quadrático

                # Backpropagation: Calcula os deltas (gradientes de erro)
                delta3 = (target - y3) * self.deriv(i3) # Delta da camada de saída
                self.w3 += self.lr * np.dot(y2.T, delta3) # Atualiza os pesos da 2ª camada para a saída

                delta2 = np.dot(delta3, self.w3.T) * self.deriv(i2)  # Delta da 2ª camada oculta
                self.w2 += self.lr * np.dot(y1.T, delta2)  # Atualiza os pesos da 1ª camada para a 2ª camada

                delta1 = np.dot(delta2, self.w2.T) * self.deriv(i1)  # Delta da 1ª camada oculta
                self.w1 += self.lr * np.dot(input_value.reshape(1, -1).T, delta1) # Atualiza os pesos da entrada para a 1ª camada

            # Calcula o MSE médio da época
            mse = sum(mse[0]) / len(x)
            
            # Exibe o MSE da época atual e a diferença em relação à época anterior
            print(f"EPOCA: {epoch} - MSE_atual: {mse} - |mse_anterior - mse_atual|: {abs(last_mse - mse)}")
            if abs(last_mse - mse) <= self.e:
                break
            
            # Armazena o MSE atual e atualiza o MSE anterior
            self.total_mse.append(mse)
            last_mse = mse
            epoch += 1
            
    # Função de teste para avaliar novos dados de entrada      
    def test(self, x):
        results = []
        for xi in x:
            input_value = np.insert(xi, 0, self.bias) # Adiciona o bias à entrada
            i1 = self.predict(input_value, self.w1)   # Soma ponderada para a 1ª camada
            y1 = self.activate(i1)                    # Ativação da 1ª camada
            i2 = self.predict(y1, self.w2)            # Soma ponderada para a 2ª camada
            y2 = self.activate(i2)                    # Ativação da 2ª camada
            i3 = self.predict(y2, self.w3)            # Soma ponderada para a camada de saída
            y3 = self.activate(i3)                    # Ativação da camada de saída

            results.append(y3[0]) # Armazena a previsão final
            
        return results

In [19]:
# Cria a rede MLP com taxa de aprendizado 0.01, tolerância 1e-7 e 3 camadas (5, 4 e 3 neurônios)
rede = MLP(lr = 0.01, e = 1e-7, neurons = [5, 4, 3])


# Treina a rede com os dados de entrada (x_treino) e suas respectivas saídas esperadas (d_treino)
rede.train(x = x_treino, d = d_treino)

EPOCA: 0 - MSE_atual: 1.4919487370214197 - |mse_anterior - mse_atual|: inf
EPOCA: 1 - MSE_atual: 1.4331691629584284 - |mse_anterior - mse_atual|: 0.05877957406299128
EPOCA: 2 - MSE_atual: 1.3692545033423154 - |mse_anterior - mse_atual|: 0.06391465961611309
EPOCA: 3 - MSE_atual: 1.3009810952573604 - |mse_anterior - mse_atual|: 0.06827340808495497
EPOCA: 4 - MSE_atual: 1.2292364281007582 - |mse_anterior - mse_atual|: 0.07174466715660222
EPOCA: 5 - MSE_atual: 1.155176163023625 - |mse_anterior - mse_atual|: 0.07406026507713315
EPOCA: 6 - MSE_atual: 1.0806067477802308 - |mse_anterior - mse_atual|: 0.07456941524339422
EPOCA: 7 - MSE_atual: 1.0082458160406085 - |mse_anterior - mse_atual|: 0.07236093173962233
EPOCA: 8 - MSE_atual: 0.9413424394942448 - |mse_anterior - mse_atual|: 0.06690337654636369
EPOCA: 9 - MSE_atual: 0.8826380867013522 - |mse_anterior - mse_atual|: 0.058704352792892545
EPOCA: 10 - MSE_atual: 0.8335340540766936 - |mse_anterior - mse_atual|: 0.04910403262465868
EPOCA: 11 - MS

In [20]:
# Submete os dados de teste à rede e armazena os resultados nas previsões
resultado = rede.test(x_teste)
resultado

[array([1.11648021e-02, 9.88984407e-01, 5.92197636e-04]),
 array([0.98529834, 0.01135828, 0.01319168]),
 array([2.11012106e-02, 9.83214303e-01, 6.82973299e-04]),
 array([0.98712514, 0.01136319, 0.01309136]),
 array([1.58397970e-02, 3.05943176e-05, 9.85815246e-01]),
 array([1.55056088e-02, 9.87289025e-01, 6.20287486e-04]),
 array([0.9775481 , 0.00856033, 0.01736892]),
 array([1.60348722e-02, 3.09573764e-05, 9.85557016e-01]),
 array([0.98634561, 0.01112702, 0.01332578]),
 array([1.54307038e-02, 3.03780079e-05, 9.85855288e-01]),
 array([1.90386207e-02, 9.83540867e-01, 6.79872295e-04]),
 array([1.41076659e-02, 9.87114165e-01, 6.24982054e-04]),
 array([1.02090869e-02, 9.89491644e-01, 5.82049439e-04]),
 array([1.08015432e-02, 9.89470817e-01, 5.82302590e-04]),
 array([1.55740796e-02, 3.03958606e-05, 9.85894455e-01]),
 array([0.98561329, 0.01089767, 0.01359315]),
 array([1.17865840e-02, 9.88927190e-01, 5.92733077e-04]),
 array([0.98742267, 0.01148447, 0.01299082])]

In [21]:
d_teste

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [22]:
# Cassifica os valores previstos
classes_resultado = classificar(resultado)

# Classifica os valores esperados
classes_esperado = classificar(d_teste)

In [23]:
print(classes_resultado)

[1, 0, 1, 0, 2, 1, 0, 2, 0, 2, 1, 1, 1, 1, 2, 0, 1, 0]


In [24]:
print(classes_esperado)

[1, 0, 1, 0, 2, 1, 0, 2, 0, 2, 1, 1, 1, 1, 2, 0, 1, 0]


In [ ]:
def acuracia(real, predito):
    acertos = 0  # Contador de acertos

    for i in range(len(real)):  # Itera sobre as listas
        if real[i] == predito[i]:  # Compara real e predito
            acertos += 1  # Incrementa acertos se forem iguais

    return acertos / len(real)  # Retorna a porcentagem de acertos

In [ ]:
acuracia(classes_esperado, classes_resultado)

1.0